In [5]:
#Importing Necessary Liberaries

import pandas as pd
import numpy as np
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
# print(openai.api_key) 

# #Reading  defects csv file
# jira_tms_data = pd.read_csv("NetCracker TMS 2023-07-22T08_01_52+0100.csv")

In [7]:
#import Langchaing and LLM liberaries
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch


# TMS Document Loading

In [12]:
# tms/GTS data 
file  = "Cleaned_NCTMSfile.csv"
tms_loader = CSVLoader(file_path=file,encoding='utf-8')
# jira_tms_data = pd.read_csv("Cleaned_NCTMSfile.csv")

In [11]:
!dir

 Volume in drive D is Extra
 Volume Serial Number is 7A36-5D60

 Directory of d:\hackathon\NC_HACKATHON\AI Chabot\solution\NCAssist

08/26/2023  08:26 PM    <DIR>          .
08/26/2023  07:56 PM    <DIR>          ..
08/26/2023  08:25 PM         5,819,663 Cleaned_NCTMSfile.csv
08/26/2023  08:11 PM            84,640 Generative_AI_for_TMS_Bassdocs_ERTdocs_JN_v1.ipynb
08/26/2023  07:46 PM            25,357 Generative_AI_for_TMS_JN_v1.ipynb
               3 File(s)      5,929,660 bytes
               2 Dir(s)  19,526,336,512 bytes free


In [13]:
tms_docs= tms_loader.load()

In [14]:
len(tms_docs)

30777

In [15]:
tms_docs[0:10]

[Document(page_content='Ticket key: TFNUK-921897\nCustom field (External Issue ID): \nPriority: Normal\nStatus: Resolved\nAssignee: hech0322\nComponent/s: MS_CIM\nFix Version/s: \nCreated: 22/Jul/23 7:39 AM\nUpdated: 22/Jul/23 7:42 AM\nResolution: Fixed\nCustom field (Root Cause Description): ', metadata={'source': 'Cleaned_NCTMSfile.csv', 'row': 0}),
 Document(page_content='Ticket key: TFNUK-921877\nCustom field (External Issue ID): INC000005928417\nPriority: Normal\nStatus: Open\nAssignee: veko0815\nComponent/s: MS_Order Entry\nFix Version/s: \nCreated: 22/Jul/23 12:34 AM\nUpdated: 22/Jul/23 6:02 AM\nResolution: \nCustom field (Root Cause Description): ', metadata={'source': 'Cleaned_NCTMSfile.csv', 'row': 1}),
 Document(page_content='Ticket key: TFNUK-921876\nCustom field (External Issue ID): INC000005915722\nPriority: Critical\nStatus: Open\nAssignee: veko0815\nComponent/s: MS_Order Entry\nFix Version/s: \nCreated: 22/Jul/23 12:17 AM\nUpdated: 22/Jul/23 6:02 AM\nResolution: \nCusto

# TMS Document splitting

In [16]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 20,
    separators=["\n\n", "\n", "(?<=\.)", " ", ""]
)

In [17]:
tms_splits = text_splitter.split_documents(tms_docs)

In [18]:
len(tms_splits)

38742

In [19]:
tms_splits[0:10]

[Document(page_content='Ticket key: TFNUK-921897\nCustom field (External Issue ID): \nPriority: Normal\nStatus: Resolved\nAssignee: hech0322\nComponent/s: MS_CIM\nFix Version/s: \nCreated: 22/Jul/23 7:39 AM\nUpdated: 22/Jul/23 7:42 AM\nResolution: Fixed\nCustom field (Root Cause Description):', metadata={'source': 'Cleaned_NCTMSfile.csv', 'row': 0}),
 Document(page_content='Ticket key: TFNUK-921877\nCustom field (External Issue ID): INC000005928417\nPriority: Normal\nStatus: Open\nAssignee: veko0815\nComponent/s: MS_Order Entry\nFix Version/s: \nCreated: 22/Jul/23 12:34 AM\nUpdated: 22/Jul/23 6:02 AM\nResolution: \nCustom field (Root Cause Description):', metadata={'source': 'Cleaned_NCTMSfile.csv', 'row': 1}),
 Document(page_content='Ticket key: TFNUK-921876\nCustom field (External Issue ID): INC000005915722\nPriority: Critical\nStatus: Open\nAssignee: veko0815\nComponent/s: MS_Order Entry\nFix Version/s: \nCreated: 22/Jul/23 12:17 AM\nUpdated: 22/Jul/23 6:02 AM\nResolution: \nCustom 

# Loading Design Document PDF

In [25]:
# bass page docs
bassdoc_loader = PyPDFLoader("O2UK-DI-ePOS-PMT-006-ScanPLUCapability-260823-0424-738.pdf") 
desing_pages = bassdoc_loader.load()

In [26]:
len(desing_pages)

26

In [27]:
desing_pages[4:]

[Document(page_content='Assumptions:\nThere will be one on one mapping between PLU and SKU for R3\nWhen the User is on the "Help" tab and he clicks "TAB" button then the cursor \nshould go to Scan PLU button (the Enter PLU field won\'t be shown, to open it the \n). click on special button is required\nIn the Scanned Product pop-up the space between Configure purchase option \nbutton and the text above it should be 32px\nPre-conditions :\nAgent can either launch ePOS for an anonymous customer through evolution  or \nlaunch ePOS from CSRD for authorized customer.\nAgent can initiate scan on Resign Options Page, Product Listing Page, Product \nDetails Page, Configuration Page and Basket Page\nExecution :\nScan/PLU Button\nEnter PLU field\nError Message when no product offering is found for the scanned PLU:', metadata={'source': 'O2UK-DI-ePOS-PMT-006-ScanPLUCapability-260823-0424-738.pdf', 'page': 4}),
 Document(page_content='Element \nnameFootnote Description Technical \ndetailsSource UI/

# Design Document splitting

In [28]:
# Split

pdf_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 20,
    separators=["\n\n", "\n", "(?<=\.)", " ", ""]
)

In [29]:
desing_pdf_splits = pdf_text_splitter.split_documents(desing_pages[4:])

In [30]:
len(desing_pdf_splits)

23

# ERT Documents Loading

In [31]:
# bass page docs
bassdoc_loader = PyPDFLoader("Rakuten_ERT - Production_Monitoring.pdf") 
ert_pages = bassdoc_loader.load()

In [32]:
len(ert_pages[3:])

33

In [33]:
ert_pages[3:]

[Document(page_content='Rakuten ERT Production Monitoring  \nProduction Monitoring with various tools  \n \n \n© 2020 NetCracker Technology Corporation.    4 \nNETCRACKER CONFIDENTIAL AND PROPRIETARY. FOR INTERNAL DISTRIBUTION ONLY.  \nDisclose and distribute solely to employees of NetCracker  Technology Corporation that have a need to know.  \n1 Introduction  \nThe objective of this document is to understand the tools used for Rakuten production \nenvironment monitoring. This document contains…  \n\uf0a7 Monitoring methods & tools  \n\uf0a7 Type of Grafana dashboards to be monitored  \n\uf0a7 ERT dashboard – high level idea  \n\uf0a7 ERT dashboard – detailing of each graph  \n\xad Understanding of each graph  \n\uf0b7 What to monitor  \n\uf0b7 How to monitor  \n\uf0b7 When to escalate  \n\uf0a7 iCinga alerts  \n\uf0a7 NewRelic alerts  ', metadata={'source': 'Rakuten_ERT - Production_Monitoring.pdf', 'page': 3}),
 Document(page_content='Rakuten ERT Production Monitoring  \nProduction M

# ERT Document Splitting

In [34]:
# Split

pdf_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    separators=["\n\n", "\n", "(?<=\.)", " ", ""]
)

In [35]:
ert_pdf_splits = pdf_text_splitter.split_documents(ert_pages[3:])

In [36]:
len(ert_pdf_splits)

47

In [37]:
ert_pdf_splits[:8]

[Document(page_content='Rakuten ERT Production Monitoring  \nProduction Monitoring with various tools  \n \n \n© 2020 NetCracker Technology Corporation.    4 \nNETCRACKER CONFIDENTIAL AND PROPRIETARY. FOR INTERNAL DISTRIBUTION ONLY.  \nDisclose and distribute solely to employees of NetCracker  Technology Corporation that have a need to know.  \n1 Introduction  \nThe objective of this document is to understand the tools used for Rakuten production \nenvironment monitoring. This document contains…  \n\uf0a7 Monitoring methods & tools  \n\uf0a7 Type of Grafana dashboards to be monitored  \n\uf0a7 ERT dashboard – high level idea  \n\uf0a7 ERT dashboard – detailing of each graph  \n\xad Understanding of each graph  \n\uf0b7 What to monitor  \n\uf0b7 How to monitor  \n\uf0b7 When to escalate  \n\uf0a7 iCinga alerts  \n\uf0a7 NewRelic alerts', metadata={'source': 'Rakuten_ERT - Production_Monitoring.pdf', 'page': 3}),
 Document(page_content='Rakuten ERT Production Monitoring  \nProduction Mon

# Combine all Docs

In [41]:
all_docs= tms_splits[0:10]+ desing_pdf_splits + ert_pdf_splits
# all_docs= jira_tms_data.head(10).to_dict('records') + desing_pdf_splits + ert_pdf_splits

In [42]:
type(all_docs)

list

# Embedding Combine Documents

In [44]:
from langchain.indexes import VectorstoreIndexCreator

In [47]:
all_index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_documents(all_docs)

# Calling LLM Chatbot

In [48]:
def main():

    while True:
        user_query = input("You: ").strip().lower()

        if user_query == "exit":
            print("Chatbot: Goodbye!")
            break
        response = all_index.query(user_query)
        print(f"Chatbot: {response}")

# Running Chatbot

In [50]:
if __name__ == '__main__':
    main()

Chatbot:  Grafana and iCinga alerts, as well as NewRelic alerts.
Chatbot:  Grafana and iCinga alerts, as well as NewRelic alerts.
Chatbot:  
- Check if any old ticket already exists for that issue 
- If ticket exists with status OPEN: update the ticket with new occurrence and details 
- If old ticket exists with status CLOSED: Reopen the ticket and update with new occurrence details 
- If ticket doesn’t exist: open a new ticket after collecting the issue details (from NewRelic tool/Greylog), logs (if required as advised by Duty manager, seek MSO help to collect the logs) 
- Consult with the duty manager about the issue 
- If issue is related to performance impact (CPU utilization is high, High DB response time, High eCare response time): Create a GTS ticket and assign to Oleg V. 
- If issue is functional (High no of business/application errors): Create a TMS ticket and assign to Yuliia S. (L2 lead). Also inform L2 TOMS person in shift about the ticket.
Chatbot:  
- Check if any old tic